# The Pandas Reference
> A tutorial on how to write clean pandas code to perform data analysis.

- toc: false 
- badges: true
- comments: true
- categories: [pandas, python]
- image: images/chart-preview.png

# About

Much of data exists in rectangular format with rows and columns. Different terms can be used to describe these kind of data   
1. Table  
2. Data frame 
3. Structured data   
4. Spreadsheets 

Pandas is one of the widely used data manipulation library in python for structured datasets. Below is a summary of the key operations that are part of any essential data analysis project(SQL equivalents). 

1. Select column references 
2. Select scalar expression  
3. Where 
4. Group By
5. Select aggregation 
6. Order By 
7. Window functions  
8. Join  


When I started using pandas, realized that there are multiple ways to perform the same operation.Also, code I was writing was not as elegant as SQL queries and hard to debug. In this blog post I will share examples of how to perform the above mentioned SQL operations in pandas and write pandas code that is readable and easy to maintain.   

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv")
pd.options.display.max_rows = 20

In [8]:
df.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#### Select columns 
Use loc with list of columns names to perform selection of columns  

```python
.loc[:,['col1','col2']] 
```

Select total_bill and tips column from the data. Note: we are using method chaining to perform operations one after another

In [8]:
(df
 .loc[:,['tip','sex']]
 .head()
)

,tip,sex
0,1.01,Female
1,1.66,Male
2,3.50,Male
3,3.31,Male
4,3.61,Female


Select only columns starting with the letter 't'. Using this simple and readable syntax enables as to perform complex select operations in pandas 

In [7]:
(df
 .loc[:,[col for col in df.columns if col.startswith('t')]]
 .head()
)

,total_bill,tip,time
0,16.99,1.01,Dinner
1,10.34,1.66,Dinner
2,21.01,3.50,Dinner
3,23.68,3.31,Dinner
4,24.59,3.61,Dinner


#### Select columns manipulation 
Use assign statement to add new columns, updated existing columns 


```python
.assign(new_col=1)
.assign(new_col=lambda x:x['col']+1)
.assign(old_col=lambda x:x['old_col']+1)
```

In [3]:
(df
 .loc[:,['total_bill','tip','sex','day','time']]
 .assign(percentage_tip=lambda x:x['tip']/x['total_bill']) #add new column
 .assign(tip=lambda x:x['tip']+1) # update existing column 
 .assign(count=1) #add constant value 
 .head()
)

,total_bill,tip,sex,day,time,percentage_tip,count
0,16.99,2.01,Female,Sun,Dinner,0.059447,1
1,10.34,2.66,Male,Sun,Dinner,0.160542,1
2,21.01,4.50,Male,Sun,Dinner,0.166587,1
3,23.68,4.31,Male,Sun,Dinner,0.139780,1
4,24.59,4.61,Female,Sun,Dinner,0.146808,1


#### Filter rows (where)
Use query to peform filting of rows in pandas  

```python
.query("col1>='10'")
.query("col1.isin(['a','b'])",engine='python')
```

In [30]:
#filter only transaction with more than 15% in tips
(df
 .loc[:,['total_bill','tip','sex','day','time']]
 .assign(percentage_tip=lambda x:x['tip']/x['total_bill'])
 .query("percentage_tip>.15")
 .head()
)

,total_bill,tip,sex,day,time,percentage_tip
1,10.34,1.66,Male,Sun,Dinner,0.160542
2,21.01,3.50,Male,Sun,Dinner,0.166587
5,25.29,4.71,Male,Sun,Dinner,0.186240
6,8.77,2.00,Male,Sun,Dinner,0.228050
9,14.78,3.23,Male,Sun,Dinner,0.218539


In [40]:
per_tip=.15
#using @ within query to refer a variable in the filter 
(df
 .loc[:,['total_bill','tip','sex','day','time']]
 .assign(percentage_tip=lambda x:x['tip']/x['total_bill'])
 .query("percentage_tip>@per_tip")
 .head()
)

,total_bill,tip,sex,day,time,percentage_tip
1,10.34,1.66,Male,Sun,Dinner,0.160542
2,21.01,3.50,Male,Sun,Dinner,0.166587
5,25.29,4.71,Male,Sun,Dinner,0.186240
6,8.77,2.00,Male,Sun,Dinner,0.228050
9,14.78,3.23,Male,Sun,Dinner,0.218539


In [5]:
#Filter only transactions happend on Sunday and Monday
(df
 .loc[:,['total_bill','tip','sex','day','time']]
 .query("day.isin(['Sun','Mon'])",engine='python')
 .head()
)

,total_bill,tip,sex,day,time
0,16.99,1.01,Female,Sun,Dinner
1,10.34,1.66,Male,Sun,Dinner
2,21.01,3.50,Male,Sun,Dinner
3,23.68,3.31,Male,Sun,Dinner
4,24.59,3.61,Female,Sun,Dinner


#### Group By and Aggregation 
Use groupby with named aggs to perform any type of aggregations 

In [41]:
#By day get average and total bill
(df
 .groupby(['day'])
 .agg(avg_bill=('total_bill','mean')
     ,total_bill=('total_bill','sum'))
 .reset_index()
)

,day,avg_bill,total_bill
0,Fri,17.151579,325.88
1,Sat,20.441379,1778.40
2,Sun,21.410000,1627.16
3,Thur,17.682742,1096.33


#### Ordering rows
Use assign statement to add new columns, updated existing columns 


```python
.sort_values(['col1','col2'],ascending=[True,False])
```

In [38]:
#By day get average and total bill.Sort the output by total_bill
(df
 .groupby(['day'])
 .agg(avg_bill=('total_bill','mean')
     ,total_bill=('total_bill','sum'))
 .reset_index()
 .sort_values(['total_bill'])
)

,day,avg_bill,total_bill
0,Fri,17.151579,325.88
3,Thur,17.682742,1096.33
2,Sun,21.410000,1627.16
1,Sat,20.441379,1778.40


### Conclusion 

In this blog post I have shared some simple tips that will improve efficiency of any data analysis projects. I plan to update this blog post with more examples to make data analysis in pandas easy. 